<center>

# **Analyse de la Danse de l'Égyptien**
</center>

### I. INSTALLATION ET UTILISATION DE LA LIBRAIRIE

Afin de garantir la fiabilité de nos futures analyses de trajectoires, nous mettons en place un test avec l'outil MMPoseInferencer. L'objectif est de vérifier la fiabilité de la lecture et la visualisation des données.

Nous ciblons ici l'extraction des articulations standards (épaules, coudes, poignets) qui seront les variables importante pour la suite de notre projet.

In [ ]:
import mmengine
import mmcv
import mmdet
import mmpose
from mmpose.apis import MMPoseInferencer
import matplotlib.pyplot as plt
import os
import json
import numpy as np
from IPython.display import HTML
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.animation as animation
from sklearn.metrics import r2_score

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inferencer = MMPoseInferencer('human', device='cpu') 

image = 'demo.jpg'
result_generator = inferencer(image, vis=True, out_dir='vis_results')
result = next(result_generator)

vis_img_path = os.path.join('vis_results', 'visualizations', os.path.basename(image))
vis_img = mmcv.imread(vis_img_path)

plt.imshow(mmcv.bgr2rgb(vis_img))
plt.axis('off')
plt.show()

On remarque bien ici les articulations du corps avec les yeux, les oreilles et le nez. Nous pouvons donc passer à la suite du projet.

### II. PRÉSENTATION DES DONNÉES UTILISÉES

Nous avons choisi de travailler sur une vidéo du jeu Just Dance. Nous avons donc deux vidéos à analyser :
- La vidéo originale de la danse 
- Une vidéo de nous qui reproduisons cette même danse

Notre but est de comparer nos mouvements par rapport à la vidéo originale pour savoir qui de nous reproduit le mieux la danse.
Nous avons choisi un mouvement assez répétitif afin de pouvoir en tirer une prédiction sur les prochains mouvements.

Nous n'avons pas encore filmé la vidéo de nous qui dansons. Pour l'instant nous allons baser nos analyses sur la vidéo originale et nous rajouterons la semaine prochaine notre vidéo.

In [ ]:
# Nous avons pris un extrait raw de just dance afin d'éviter le bruit sur la vidéo

HTML("""
<video width="600" controls>
  <source src="original_30fps.mp4" type="video/mp4">
</video>
""")

### PREMIERS RÉSULTATS

Utilisons MMPoseInferencer afin d'extraire les keypoints de nos vidéos.

In [ ]:
original = 'original_30fps.mp4'

result_generator = inferencer(original, vis=True, out_dir='vis_results')

for frame_result in result_generator:
    pass

L'outil nous génère deux fichiers : 
- Un fichier vidéo mp4 qui reprend la vidéo de base en ajoutant les keypoints directement sur l'image
- Un fichier json qui contient toutes les coordonnées de chaque keypoints image par image et personne par personne

C'est sur ce dernier fichier json que nous allons pouvoir étudier les trajectoires et mouvements de nos personnages.

In [ ]:
HTML(""" 
<video width="600" controls>
  <source src="vis_results/visualizations/original.mp4" type="video/mp4">
</video>
""")

BATT

### Battle de danse 

In [ ]:
HTML("""
<video width="600" controls>
  <source src="slayeuse.mp4" type="video/mp4">
</video>
""")

In [ ]:
original = 'slayeuse.mp4'
result_generator = inferencer(original, vis=True, out_dir='vis_results')
for frame_result in result_generator:
    pass

In [ ]:
import json
import numpy as np

def normaliser_squelette(kp):
    # Centre sur le bassin et met à l'échelle
    hip_center = (kp[11] + kp[12]) / 2
    kp_centered = kp - hip_center
    shoulder_center = (kp_centered[5] + kp_centered[6]) / 2
    torso_size = np.linalg.norm(shoulder_center)
    if torso_size == 0: return kp_centered
    return kp_centered / torso_size

def charger_reference(fichier_ref):
    # Charge le fichier original
    with open(fichier_ref, 'r') as f:
        data = json.load(f)
    trajectoire = []
    for frame in data:
        if len(frame['instances']) > 0:
            kps = np.array(frame['instances'][0]['keypoints'])
            if kps.shape[1] == 3: kps = kps[:, :2]
            trajectoire.append(normaliser_squelette(kps))
    return np.array(trajectoire)

def extraire_groupe(fichier_json_groupe, nb_personnes=4):
    # Charge le fichier du groupeet trie de gauche à droite
    with open(fichier_json_groupe, 'r') as f:
        data = json.load(f)
    trajectoires = [[] for _ in range(nb_personnes)]
    
    for frame in data:
        instances = frame.get('instances', [])
        if len(instances) < nb_personnes: continue 
        
        # Récupération et Tri (Gauche -> Droite)
        personnes_brutes = []
        for inst in instances:
            kps = np.array(inst['keypoints'])
            if kps.shape[1] == 3: kps = kps[:, :2]
            personnes_brutes.append(kps)
        
        personnes_triees = sorted(personnes_brutes, key=lambda p: np.mean(p[:, 0]))
        personnes_triees = personnes_triees[:nb_personnes]
        
        for i, kps in enumerate(personnes_triees):
            trajectoires[i].append(normaliser_squelette(kps))
            
    return [np.array(t) for t in trajectoires]

def calculer_score_similitude(ref, student):
    n = min(len(ref), len(student))
    if n == 0: return 0.0
    
    ref = ref[:n]
    stud = student[:n]
    
    diff = ref - stud
    erreur_moyenne = np.mean(np.linalg.norm(diff, axis=(1,2)))
    facteur_severite = 0.3 
    
    score_final = 100 * np.exp(-facteur_severite * erreur_moyenne)
    return score_final
    
data_ref = charger_reference('vis_results/predictions/original.json')
danseuses = extraire_groupe('vis_results/predictions/slayeuse.json', nb_personnes=4)

noms = ["Gauche", "Milieu-Gauche", "Milieu-Droite", "Droite"]
resultats = {}

for i in range(4):
    if len(danseuses[i]) > 0:
        score = calculer_score_similitude(data_ref, danseuses[i])
        resultats[noms[i]] = score
    else:
        print(f" Pas assez de données détectées pour {noms[i]}")

print("\n===  RÉSULTATS BATTLE DE DANSE  ===")
# reverse=True pour avoir le 1er (le plus haut score) en haut
for nom, sc in sorted(resultats.items(), key=lambda x: x[1], reverse=True):
    print(f"{nom} : {sc:.1f} % de réussite")

### III. ANALYSE DES TRAJECTOIRES

Nous allons utiliser le fichier json pour extraire la positions des poignets du danseur frame par frame. Cependant, les coordonnées brutes extraites par MMPose sont exprimées en pixels. Nous ne pouvons donc pas travailler directement avec ces valeurs car :
- Un danseur proche de l'objectif aura des mouvements qui paraissent "plus grands" qu'un danseur au second plan.
- La taille des membres varie d'un individu à l'autre.

Pour l'instant cela ne pose pas de problème car nous avons qu'une seule vidéo à analyser mais quand nous ajouterons l'analyse de notre danse, nous aurons besoin de données normalisées.

Nous déplaçons donc l'origine du repère au milieu de la ligne des épaules. Ainsi, la position du poignet est mesurée relativement au buste et non au cadre de l'image.

La formule appliquée pour chaque frame est :

$$Y_{norm} = \frac{Y_{poignet} - Y_{milieu\_épaules}}{Distance(Épaule_{G}, Épaule_{D})}$$

In [ ]:
def trajectoires_normalisees(nom_fichier, nb_danseurs=1):
    with open(nom_fichier, 'r') as f:
        data = json.load(f)
    
    frames = []
    # On prépare des listes pour stocker les poignets droits et gauches
    traj_droit = [[] for _ in range(nb_danseurs)]
    traj_gauche = [[] for _ in range(nb_danseurs)]
    
    for f_data in data:
        frames.append(f_data['frame_id'])
        # Tri des personnes de gauche à droite pour ne pas mélanger les courbes
        insts = sorted(f_data['instances'], key=lambda x: np.mean([kp[0] for kp in x['keypoints']]))
        
        for i in range(nb_danseurs): #on tente d'éliminer les "fantômes" en saisissant manuellement le nombre supposé de danseurs dans nb_danseurs
            if i < len(insts):
                keypoints = np.array(insts[i]['keypoints'])
                
                # 1. Calcul du milieu des épaules (référence 0)
                milieu_epaules = (keypoints[5][1] + keypoints[6][1]) / 2 #standard COCO human keypoints : 5=épaules gauche, 6=épaule droite
                
                # 2. Calcul de la largeur des épaules (unité de normalisation)
                largeur_epaules = np.linalg.norm(np.array(keypoints[5][:2]) - np.array(keypoints[6][:2]))
                largeur_epaules = largeur_epaules if largeur_epaules > 0 else 1
                
                # 3. Calcul de la position relative (Y_poignet - Y_épaules) / Largeur_Épaules
                traj_droit[i].append((keypoints[10][1] - milieu_epaules) / largeur_epaules) #10=poignet droit
                traj_gauche[i].append((keypoints[9][1] - milieu_epaules) / largeur_epaules) #9=poignet gauche
            else:
                traj_droit[i].append(None)
                traj_gauche[i].append(None)
                
    return frames, traj_droit, traj_gauche


frames, poignet_droit, poignet_gauche = trajectoires_normalisees('vis_results/predictions/original.json', nb_danseurs=1)

plt.figure(figsize=(14, 8))
plt.plot(frames, poignet_droit[0], color='blue', linewidth=2, label='Poignet droit')
plt.plot(frames, poignet_gauche[0], color='green', linewidth=2, label='Poignet gauche')
plt.axhline(0, color='red', linestyle='-', linewidth=0.8, label='Niveau Épaules')
plt.title("Analyse des poignets gauche et droite")
plt.xlabel("Frames")
plt.ylabel("Distance relative (Largeurs d'épaules)")
plt.gca().invert_yaxis() 
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize='small')
plt.grid(True, alpha=0.15)
plt.tight_layout()

plt.show()

On remarque bien ici le mouvement des poignets par rapport aux épaules, c'est un mouvement répétitif et symétrique par rapport aux épaules ce qui va faciliter la prédiction. La ligne d'épaule ne bouge pas puisque c'est l'origine du repère.On observe ici uniquement le mouvement des poignets par rapport aux épaules et non par rapport à la caméra.

 ### IV. PRÉDICTION DES PROCHAINS MOUVEMENTS

Notre stratégie de modélisation se décomposera en deux temps :

- Approche supervisée : implémentation d'un réseau récurrent type LSTM pour modéliser la dynamique temporelle de la danse. Ce modèle servira de référence pour la prédiction de trajectoires courtes.

- Apprentissage par renforcement : utilisation de l'algorithme PPO (Proximal Policy Optimization) pour entraîner un agent virtuel à faire la danse de l'égyptien. L'environnement sera défini par la distance entre l'agent et la trajectoire normalisée du fichier original.json. Cette approche permettra de tester la robustesse du mouvement face à des perturbations.

#### IV.1) APPROCHE SUPERVISÉE

Le but ici est de prédire la frame t+1 en fonction des frames précédentes. Nous allons utiliser la MSE comme loss.

Tout d'abbord, nous devons normaliser tous les points par rapport aux centre des épaules, jusqu'ici, nous avions étudié seulement les poignets mais pour la prédiction nous avons besoin de tous les keypoints.

In [ ]:
def prepare_stable_data(file_name, window_size=45):
    with open(file_name, 'r') as f:
        data = json.load(f)
    
    all_frames_data = []
    for f_data in data:
        if len(f_data['instances']) > 0:
            kp = np.array(f_data['instances'][0]['keypoints']) 
            # On centre sur les épaules
            center = (kp[5] + kp[6]) / 2
            # On normalise la taille
            scale = np.linalg.norm(kp[5] - kp[6])
            if scale == 0: scale = 1.0
            norm_kp = (kp - center) / scale
            all_frames_data.append(norm_kp.flatten())
    
    all_frames_data = np.array(all_frames_data)
    
    X, y = [], []
    for i in range(len(all_frames_data) - window_size):
        X.append(all_frames_data[i : i + window_size]) #toutes les frames sauf la dernière
        y.append(all_frames_data[i + window_size]) # seulement la dernière frame
        
    return np.array(X), np.array(y), all_frames_data

# On utilise window_size=30 pour un bon compromis mémoire/stabilité
X_train, y_train, full_pos = prepare_stable_data('vis_results/predictions/original.json', window_size=20)

X_train_tensor = torch.from_numpy(X_train).float().to(device)
y_train_tensor = torch.from_numpy(y_train).float().to(device)

print(f"OK ! Données prêtes. X: {X_train.shape}, y: {y_train.shape}")

In [ ]:

class PosePredictor(nn.Module):
    def __init__(self, input_dim, hidden_dim=512, num_layers=2):
        super(PosePredictor, self).__init__()
        
        # On augmente hidden_dim à 512 pour plus de "mémoire" de la danse
        # On ajoute dropout=0.2 pour éviter qu'il apprenne par cœur et qu'il bloque
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout=0.2)
        
        self.fc = nn.Linear(hidden_dim, input_dim)
        
    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :] 
        out = self.fc(out)
        return out

In [ ]:


model = PosePredictor(input_dim=34, hidden_dim=512, num_layers=2).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

epochs = 600 # On double les époques pour comprendre le rythme
loss_history = []

for epoch in range(epochs):
    model.train()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    loss_history.append(loss.item())
    
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.6f}')

In [ ]:

model.eval()
input_seq = X_train_tensor[-1:]
current_pos = full_pos[-1]

generated_frames = []

alpha = 1.75 # Plus ce chiffre est grand plus le modele va forcer ses mouvements

nb_frame = 30
with torch.no_grad():
    for _ in range(nb_frame):
        # On predit la pose "prudente"
        pred_pose = model(input_seq).cpu().numpy().squeeze()
        
        # On calcule le mouvement prévu par rapport à la frame précédente
        movement = pred_pose - input_seq[0, -1, :].cpu().numpy()
        
        # On rajoute ce mouvement multiplié par alpha à la dernière frame
        boosted_pose = input_seq[0, -1, :].cpu().numpy() + (movement * alpha)
        
        # Pour que les bras ne s'allongent pas trop
        boosted_pose = np.clip(boosted_pose, -3, 6)
        
        generated_frames.append(boosted_pose)
        
        # Mise à jour
        new_frame = torch.from_numpy(boosted_pose).float().to(device).view(1, 1, 34)
        input_seq = torch.cat((input_seq[:, 1:, :], new_frame), dim=1)

generated_frames = np.array(generated_frames)

In [ ]:


def final_dance_show(full_pos, generated_frames):
    # On prend les 30 dernières frames du vrai danseur
    real_part = full_pos[-nb_frame:]
    
    # On applique le modèle
    all_dance = np.concatenate([real_part, generated_frames])
    
    fig, ax = plt.subplots(figsize=(7, 7), facecolor='black')
    
    def update(i):
        ax.clear()
        ax.set_facecolor('black')
        ax.set_xlim(-5, 5)
        ax.set_ylim(10, -5) 
        ax.axis('off')
        
        kp = all_dance[i].reshape(17, 2)
        color = '#00FF00' if i < nb_frame else '#00FFFF' 
        
        links = [(5,6),(5,7),(7,9),(6,8),(8,10),(5,11),(6,12),(11,12),(11,13),(13,15),(12,14),(14,16)]
        for s, e in links:
            ax.plot([kp[s,0], kp[e,0]], [kp[s,1], kp[e,1]], color=color, lw=3)
        ax.scatter(kp[:,0], kp[:,1], c='white', s=20)
        ax.set_title("RÉEL (VERT) puis IA GÉNÉRÉE (CYAN)", color='white')

    ani = animation.FuncAnimation(fig, update, frames=len(all_dance), interval=33)
    plt.close()
    return ani

HTML(final_dance_show(full_pos, generated_frames).to_jshtml())

In [ ]:
score_r2 = r2_score(full_pos[-nb_frame:].flatten(), generated_frames.flatten())
print(score_r2)